In [0]:
%tensorflow_version 1.x

In [0]:
!pip install ekphrasis

In [0]:
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [0]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


In [0]:
def print_text(texts,i,j):
    for u in range(i,j):
        print(texts[u])
        print()

In [0]:
df = pd.read_csv("/content/drive/My Drive/Memotion /memotion_dataset_7k/labels.csv", header=None, skiprows=1)
text_array = df[3]
image_path = df[1]
print("Length of training data:",len(text_array))
print("Total images:",len(image_path))

In [0]:
df_val = pd.read_csv("/content/drive/My Drive/semeval-2020_trialdata/data1.csv", header=None, skiprows=1)
text_array_val = df_val[2]
image_path_val = df_val[0]
print("Length of validation data:",len(text_array_val))
print("No. if memes in validation set:",len(image_path_val))

**Retrieving embedding vectors for the images:**

In [0]:
# Loading image encodings
def map_func(img_name):
    encoding = np.load("/content/drive/My Drive/Memotion /resnet_train/"+img_name +'.npy')
    return np.asarray(encoding)

image_encodings = np.zeros((len(image_path),2048))
for index in range(0,len(image_path)):
    if(index%100 == 0):
        print(str(index)+"th entry done....")
    image_encodings[index] = map_func(image_path[index])
    
print("Images after getting encoded from resnet:")
print(np.shape(image_encodings))

In [0]:
print("Size of text array: " + str(np.size(text_array)))
print("Size of image embeddings: " + str(np.shape(image_encodings)))
index = 0
print("Image name: " + image_path[index])
print("Image text: " + text_array[index])
print("Size of image encoding: " + str(np.shape(image_encodings[index])))
print("Image embedding: " )
print(image_encodings[index])

In [0]:
def map_func(img_name):
    encoding = np.load("/content/drive/My Drive/Memotion /resnet_trial/"+img_name +'.npy')
    return np.asarray(encoding)

image_encodings_val = np.zeros((len(image_path_val),2048))
for index in range(0,len(image_path_val)):
    if(index%100 == 0):
        print(str(index)+"th entry done....")
    image_encodings_val[index] = map_func(image_path_val[index])
    
print("Images after getting encoded from resnet:")
print(np.shape(image_encodings_val))

In [0]:
print("Size of validation text array: " + str(np.size(text_array_val)))
print("Size of validation image embeddings: " + str(np.shape(image_encodings_val)))
index = 0
print("Image name: " + image_path_val[index])
print("Image text: " + text_array_val[index])
print("Size of image encoding: " + str(np.shape(image_encodings_val[index])))
print("Image embedding: " )
print(image_encodings_val[index])

<h3> Pre-processing </h3>

In [0]:
# Removing nan from train data
# Eg. Index 29 contained a NaN value, we replace that with a none keyword
flag = text_array.isnull()
for i in range(0,len(text_array)):
  if flag[i]:
      text_array[i] = "none"
print(text_array[4799])

In [0]:
# Removing nan from trial data
# Eg. Index 29 contained a NaN value, we replace that with a none keyword
flag_val = text_array_val.isnull()
for i in range(0,len(text_array_val)):
  if flag_val[i]:
      text_array_val[i] = "none"
print(text_array_val[29])

In [0]:
#removing website names
def remove_website(text):
    return " ".join([word if re.search("r'https?://\S+|www\.\S+'|((?i).com$|.co|.net)",word)==None else "" for word in text.split(" ") ])

# Training set 
text_array = text_array.apply(lambda text: remove_website(text))
print_text(text_array,99,101)

print("#########################################################################")

# Validation set
text_array_val = text_array_val.apply(lambda text: remove_website(text))
print_text(text_array_val,99,101)

In [0]:
# Functions for chat word conversion
f = open("/content/drive/My Drive/Memotion /slang.txt", "r")
chat_words_str = f.read()
chat_words_map_dict = {}
chat_words_list = []

for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)


In [0]:
# Chat word conversion
# Training set
text_array = text_array.apply(lambda text: chat_words_conversion(text))
print_text(text_array,98,100)

print("#########################################################################")

# Validation set
text_array_val = text_array_val.apply(lambda text: chat_words_conversion(text))
print_text(text_array_val,98,100)

In [0]:
#Function for emoticon conversion
from emoticons import EMOTICONS

def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', " ".join(EMOTICONS[emot].replace(",","").split()), text)
    return text


#testing the emoticon function
text = "Hello :-) :-)"
text = convert_emoticons(text)
print(text + "\n")

In [0]:
# Emoticon conversion

# Training set
text_array = text_array.apply(lambda text: convert_emoticons(text))
print_text(text_array,0,5)

print("##########################################################################")

# Validation set
text_array_val = text_array_val.apply(lambda text: convert_emoticons(text))
print_text(text_array_val,0,5)

In [0]:
# Ekphrasis pipe for text pre-processing
def ekphrasis_pipe(sentence):
    cleaned_sentence = " ".join(text_processor.pre_process_doc(sentence))
    return cleaned_sentence

# Training set
text_array = text_array.apply(lambda text: ekphrasis_pipe(text))
# Validation set
text_array_val = text_array_val.apply(lambda text: ekphrasis_pipe(text))

In [0]:
# Testing validation data after ekphrasis pipe
print_text(text_array_val,110,115)

In [0]:
# Removing unnecessary punctuations
PUNCT_TO_REMOVE = "\"$%&'()+,-./;=[\]^_`{|}~"
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

# Training set
text_array = text_array.apply(lambda text: remove_punctuation(text))
# Validation set
text_array_val = text_array_val.apply(lambda text: remove_punctuation(text))

<h2>Text Pre processing complete</h2>
<p>Data stored in text_array</p>
<p>Validation data in text_array_val</p>

In [0]:
# Finding length of longest array
maxLen = len(max(text_array,key = lambda text: len(text.split(" "))).split(" "))
print(maxLen)

In [0]:
# Printing the length longest N sentences  
u = lambda text: len(text.split(" "))
sentence_lengths = []
for x in text_array:
    sentence_lengths.append(u(x))
print(sorted(sentence_lengths)[-70:])
print(len(sentence_lengths))

<h2>Reading glove values</h2>

In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map, words

In [0]:
# We need word_to_index and index_to_word for creating the embedding layer.
word_to_index, index_to_word, word_to_vec_map, vocab = read_glove_vecs('/content/drive/My Drive/Memotion /glove.twitter.27B.100d.txt')

In [0]:
# Function for rectifying the error in column values. Some rows were shifted to left by 1 position
for i in range(0,len(df_val[8])):
  if df_val[8][i] == "image_and_text " or df_val[8][i] == "image" or df_val[8][i] == "text":
    df_val[8][i] = df_val[7][i]
    df_val[7][i] = df_val[6][i]
    df_val[6][i] = df_val[5][i]
    df_val[5][i] = df_val[4][i]
    df_val[4][i] = df_val[3][i]

In [0]:
# Count of each label in dataset
from collections import Counter

# Training set states for Subtask A
state = np.asarray(df[8])
state_val = np.asarray(df_val[8])

# Training set states for humour
state_humour = np.asarray(df[4])
state_humour_val = np.asarray(df_val[4])

# Training set states for sarcasm
state_sarcasm = np.asarray(df[5])
state_sarcasm_val = np.asarray(df_val[5])

# Training set states for offensive
state_offense = np.asarray(df[6])
state_offense_val = np.asarray(df_val[6])

# Training set states for motivation
state_motivate = np.asarray(df[7])
state_motivate_val = np.asarray(df_val[7])

# Printing counts for analysis
# Normal states
print("Elements: ",set(state))
print("Length: ",len(state))
print(Counter(state))
print("##########################################################################")
print("Elements: ",set(state_val))
print("Length: ",len(state_val))
print(Counter(state_val))

# Humour states
print("\n****************************************************************************************************\n")
print("Elements: ",set(state_humour))
print("Length: ",len(state_humour))
print(Counter(state_humour))
print("##########################################################################")
print("Elements: ",set(state_humour_val))
print("Length: ",len(state_humour_val))
print(Counter(state_humour_val))

# Sarcasm states
print("\n****************************************************************************************************\n")
print("Elements: ",set(state_sarcasm))
print("Length: ",len(state_sarcasm))
print(Counter(state_sarcasm))
print("##########################################################################")
print("Elements: ",set(state_sarcasm_val))
print("Length: ",len(state_sarcasm_val))
print(Counter(state_sarcasm_val))

# Offense states
print("\n****************************************************************************************************\n")
print("Elements: ",set(state_offense))
print("Length: ",len(state_offense))
print(Counter(state_offense))
print("##########################################################################")
print("Elements: ",set(state_offense_val))
print("Length: ",len(state_offense_val))
print(Counter(state_offense_val))

# Motivate states
print("\n****************************************************************************************************\n")
print("Elements: ",set(state_motivate))
print("Length: ",len(state_motivate))
print(Counter(state_motivate))
print("##########################################################################")
print("Elements: ",set(state_motivate_val))
print("Length: ",len(state_motivate_val))
print(Counter(state_motivate_val))

In [0]:
# Subtask A
# Converting the Labels to integer values for conversion to one hot vectors
Y = []
Y_val = []

Yn = []
Yn_val = []

# Training set    
for i in range(0,len(state)):
    if(state[i] == "neutral"):
        Y.append(0)
        Yn.append(0)
    if(state[i] == "positive"):
        Y.append(1)
        Yn.append(1)
    if(state[i] == "very_positive"):
        Y.append(1)
        Yn.append(2)
    if(state[i] == "negative"):
        Y.append(2)
        Yn.append(3)
    if(state[i] == "very_negative"):
        Y.append(2)
        Yn.append(4)

# Validation set
for i in range(0,914):
    if(state_val[i] == "neutral"):
        Y_val.append(0)
        Yn_val.append(0)
    if(state_val[i] == "positive"):
        Y_val.append(1)
        Yn_val.append(1)
    if(state_val[i] == "very_positive"):
        Y_val.append(1)
        Yn_val.append(2)
    if(state_val[i] == "negative"):
        Y_val.append(2)
        Yn_val.append(3)
    if(state_val[i] == "very_negative"):
        Y_val.append(2)
        Yn_val.append(4)

In [0]:
print(Counter(state))
print(Counter(Y),len(Y))
print(Counter(Yn),len(Yn))

In [0]:
# Subtask B and Subtask C ---------- Humour
# Converting the Labels to integer values for conversion to one hot vectors
Y_humour = []
Y_humour_val = []

Yn_humour = []
Yn_humour_val = []

Ynf_humour = []
Ynf_humour_val = []

# 'not_funny', 'hilarious', 'funny', 'very_funny'
# Training set    
for i in range(0,len(state)):
    if(state_humour[i] == "not_funny"):
        Y_humour.append(0)
        Yn_humour.append(0)
        Ynf_humour.append(0)
    if(state_humour[i] == "funny"):
        Y_humour.append(1)
        Yn_humour.append(1)
        Ynf_humour.append(1)
    if(state_humour[i] == "very_funny" or state_humour[i] == "hilarious"):
        Y_humour.append(2)
        Ynf_humour.append(1)
        if(state_humour[i] == "very_funny"):
          Yn_humour.append(2)
        else:
          Yn_humour.append(3)

# Validation set
for i in range(0,914):
    if(state_humour_val[i] == "not_funny"):
        Y_humour_val.append(0)
        Yn_humour_val.append(0)
        Ynf_humour_val.append(0)
    if(state_humour_val[i] == "funny"):
        Y_humour_val.append(1)
        Yn_humour_val.append(1)
        Ynf_humour_val.append(1)
    if(state_humour_val[i] == "very_funny" or state_humour_val[i] == "hilarious"):
        Y_humour_val.append(2)
        Ynf_humour_val.append(1)
        if(state_humour_val[i] == "very_funny"):
          Yn_humour_val.append(2)
        else:
          Yn_humour_val.append(3)

In [0]:
print(Counter(state_humour))
print(Counter(Y_humour),len(Y_humour))
print(Counter(Yn_humour),len(Yn_humour))
print(Counter(Ynf_humour),len(Ynf_humour))

In [0]:
# Subtask B and Subtask C ---------- Sarcasm
# Converting the Labels to integer values for conversion to one hot vectors
Y_sarcasm = []
Y_sarcasm_val = []

Yn_sarcasm = []
Yn_sarcasm_val = []

Ynf_sarcasm = []
Ynf_sarcasm_val = []

# 'general', 'not_sarcastic', 'twisted_meaning', 'very_twisted'
# Training set    
for i in range(0,len(state)):
    if(state_sarcasm[i] == "not_sarcastic"):
        Y_sarcasm.append(0)
        Yn_sarcasm.append(0)
        Ynf_sarcasm.append(0)
    if(state_sarcasm[i] == "general"):
        Y_sarcasm.append(1)
        Yn_sarcasm.append(1)
        Ynf_sarcasm.append(1)
    if(state_sarcasm[i] == "very_twisted" or state_sarcasm[i] == "twisted_meaning"):
        Y_sarcasm.append(2)
        Ynf_sarcasm.append(1)
        if(state_sarcasm[i] == "twisted_meaning"):
          Yn_sarcasm.append(2)
        else:
          Yn_sarcasm.append(3)

# Validation set
for i in range(0,914):
    if(state_sarcasm_val[i] == "not_sarcastic"):
        Y_sarcasm_val.append(0)
        Yn_sarcasm_val.append(0)
        Ynf_sarcasm_val.append(0)
    if(state_sarcasm_val[i] == "general"):
        Y_sarcasm_val.append(1)
        Yn_sarcasm_val.append(1)
        Ynf_sarcasm_val.append(1)
    if(state_sarcasm_val[i] == "very_twisted" or state_sarcasm_val[i] == "twisted_meaning"):
        Y_sarcasm_val.append(2)
        Ynf_sarcasm_val.append(1)
        if(state_sarcasm_val[i] == "twisted_meaning"):
          Yn_sarcasm_val.append(2)
        else:
          Yn_sarcasm_val.append(3)

In [0]:
print(Counter(state_sarcasm))
print(Counter(Y_sarcasm),len(Y_sarcasm))
print(Counter(Yn_sarcasm),len(Yn_sarcasm))
print(Counter(Ynf_sarcasm),len(Ynf_sarcasm))

In [0]:
# Subtask B and Subtask C ---------- Offense
# Converting the Labels to integer values for conversion to one hot vectors
Y_offense = []
Y_offense_val = []

Yn_offense = []
Yn_offense_val = []

Ynf_offense = []
Ynf_offense_val = []

# 'very_offensive', 'not_offensive', 'slight', 'hateful_offensive'
# Training set    
for i in range(0,len(state)):
    if(state_offense[i] == "not_offensive"):
        Y_offense.append(0)
        Yn_offense.append(0)
        Ynf_offense.append(0)
    if(state_offense[i] == "slight"):
        Y_offense.append(1)
        Yn_offense.append(1)
        Ynf_offense.append(1)
    if(state_offense[i] == "very_offensive" or state_offense[i] == "hateful_offensive"):
        Y_offense.append(2)
        Ynf_offense.append(1)
        if(state_offense[i] == "very_offensive"):
          Yn_offense.append(2)
        else:
          Yn_offense.append(3)

# Validation set
for i in range(0,914):
    if(state_offense_val[i] == "not_offensive"):
        Y_offense_val.append(0)
        Yn_offense_val.append(0)
        Ynf_offense_val.append(0)
    if(state_offense_val[i] == "slight"):
        Y_offense_val.append(1)
        Yn_offense_val.append(1)
        Ynf_offense_val.append(1)
    if(state_offense_val[i] == "very_offensive" or state_offense_val[i] == "hateful_offensive"):
        Y_offense_val.append(2)
        Ynf_offense_val.append(1)
        if(state_offense_val[i] == "very_offensive"):
          Yn_offense_val.append(2)
        else:
          Yn_offense_val.append(3)

In [0]:
print(Counter(state_offense))
print(Counter(Y_offense),len(Y_offense))
print(Counter(Yn_offense),len(Yn_offense))
print(Counter(Ynf_offense),len(Ynf_offense))

In [0]:
# Subtask B and Subtask C ---------- Motivate
# Converting the Labels to integer values for conversion to one hot vectors
Y_motivate = []
Y_motivate_val = []

# 'not_motivational', 'motivational'
# Training set    
for i in range(0,len(state)):
    if(state_motivate[i] == "motivational"):
      Y_motivate.append(1)
    else:
      Y_motivate.append(0)

# Validation set
for i in range(0,914):
    if(state_motivate_val[i] == "motivational"):
      Y_motivate_val.append(1)
    else:
      Y_motivate_val.append(0)

In [0]:
print(Counter(state_motivate))
print(Counter(Y_motivate),len(Y_motivate))

In [0]:
# Testing the conversion into integers
for i in range(0,5):
  print(text_array_val[i])
  print(state_val[i],Yn_val[i])

In [0]:
# Verifying train set 
X = np.asarray(list(text_array))

# Subtask A
Y = np.asarray(list(Y))
Yn = np.asarray(list(Yn))

# Subtask B,C --- Humour
Y_humour = np.asarray(list(Y_humour))
Yn_humour = np.asarray(list(Yn_humour))
Ynf_humour = np.asarray(list(Ynf_humour))

# Subtask B,C --- Sarcasm
Y_sarcasm = np.asarray(list(Y_sarcasm))
Yn_sarcasm = np.asarray(list(Yn_sarcasm))
Ynf_sarcasm = np.asarray(list(Ynf_sarcasm))

# Subtask B,C --- Offense
Y_offense = np.asarray(list(Y_offense))
Yn_offense = np.asarray(list(Yn_offense))
Ynf_offense = np.asarray(list(Ynf_offense))

# Subtask B,C --- Motivation
Y_motivate = np.asarray(list(Y_motivate))

print(type(X))
print(type(image_encodings))
print(np.shape(X),np.shape(Y),np.shape(image_encodings))

# Verifying validation set ------------------------------
X_val = np.asarray(list(text_array_val))

# Subtask A
Y_val = np.asarray(list(Y_val))
Yn_val = np.asarray(list(Yn_val))

# Subtask B,C --- Humour
Y_humour_val = np.asarray(list(Y_humour_val))
Yn_humour_val = np.asarray(list(Yn_humour_val))
Ynf_humour_val = np.asarray(list(Ynf_humour_val))

# Subtask B,C --- Sarcasm
Y_sarcasm_val = np.asarray(list(Y_sarcasm_val))
Yn_sarcasm_val = np.asarray(list(Yn_sarcasm_val))
Ynf_sarcasm_val = np.asarray(list(Ynf_sarcasm_val))

# Subtask B,C --- Offense
Y_offense_val = np.asarray(list(Y_offense_val))
Yn_offense_val = np.asarray(list(Yn_offense_val))
Ynf_offense_val = np.asarray(list(Ynf_offense_val))

# Subtask B,C --- Motivation
Y_motivate_val = np.asarray(list(Y_motivate_val))

print(type(X_val))
print(type(image_encodings_val))
print(np.shape(X_val),np.shape(Y_val),np.shape(image_encodings_val))

<h3>Shuffling training and validation data</h3>

In [0]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [0]:
X, image_encodings, state, state_humour, state_sarcasm, state_offense, state_motivate, Y, Yn, Y_humour, Yn_humour, Ynf_humour, Y_sarcasm, Yn_sarcasm, Ynf_sarcasm, Y_offense, Yn_offense, Ynf_offense, Y_motivate = shuffle(X, image_encodings, state, state_humour, state_sarcasm, state_offense, state_motivate, Y, Yn, Y_humour, Yn_humour, Ynf_humour, Y_sarcasm, Yn_sarcasm, Ynf_sarcasm, Y_offense, Yn_offense, Ynf_offense, Y_motivate, random_state=0)
X, image_encodings, state, state_humour, state_sarcasm, state_offense, state_motivate, Y, Yn, Y_humour, Yn_humour, Ynf_humour, Y_sarcasm, Yn_sarcasm, Ynf_sarcasm, Y_offense, Yn_offense, Ynf_offense, Y_motivate = shuffle(X, image_encodings, state, state_humour, state_sarcasm, state_offense, state_motivate, Y, Yn, Y_humour, Yn_humour, Ynf_humour, Y_sarcasm, Yn_sarcasm, Ynf_sarcasm, Y_offense, Yn_offense, Ynf_offense, Y_motivate, random_state=1)
X, image_encodings, state, state_humour, state_sarcasm, state_offense, state_motivate, Y, Yn, Y_humour, Yn_humour, Ynf_humour, Y_sarcasm, Yn_sarcasm, Ynf_sarcasm, Y_offense, Yn_offense, Ynf_offense, Y_motivate = shuffle(X, image_encodings, state, state_humour, state_sarcasm, state_offense, state_motivate, Y, Yn, Y_humour, Yn_humour, Ynf_humour, Y_sarcasm, Yn_sarcasm, Ynf_sarcasm, Y_offense, Yn_offense, Ynf_offense, Y_motivate, random_state=44)

In [0]:
#Testing after Shuffling
for i in range(40,50):
  print(X[i])
  print(state_humour[i],":",Y_humour[i],Yn_humour[i],Ynf_humour[i])
print(Counter(state_humour))

In [0]:
print(type(X))
print(type(Y))
print(np.shape(X),np.shape(Y),np.shape(image_encodings))
print(np.shape(X_val),np.shape(Y_val),np.shape(image_encodings_val))

In [0]:
# Converting to one hot vectors
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)] #u[Y] helps to index each element of Y index at u. U here is a class array
    return Y

In [0]:
# Subtask A
Y_oh_train = convert_to_one_hot(np.array(Y), C = 3)
Y_oh_val = convert_to_one_hot(np.array(Y_val), C = 3)
Y_oh_n_train = convert_to_one_hot(np.array(Yn), C = 5)
Y_oh_n_val = convert_to_one_hot(np.array(Yn_val), C = 5)

# Subtask B,C ---  Humour
Y_oh_humour_train = convert_to_one_hot(np.array(Y_humour), C = 3)
Y_oh_humour_val = convert_to_one_hot(np.array(Y_humour_val), C = 3)
Y_oh_n_humour_train = convert_to_one_hot(np.array(Yn_humour), C = 4)
Y_oh_n_humour_val = convert_to_one_hot(np.array(Yn_humour_val), C = 4)

# Subtask B,C ---  Sarcasm
Y_oh_sarcasm_train = convert_to_one_hot(np.array(Y_sarcasm), C = 3)
Y_oh_sarcasm_val = convert_to_one_hot(np.array(Y_sarcasm_val), C = 3)
Y_oh_n_sarcasm_train = convert_to_one_hot(np.array(Yn_sarcasm), C = 4)
Y_oh_n_sarcasm_val = convert_to_one_hot(np.array(Yn_sarcasm_val), C = 4)

# Subtask B,C ---  Offense
Y_oh_offense_train = convert_to_one_hot(np.array(Y_offense), C = 3)
Y_oh_offense_val = convert_to_one_hot(np.array(Y_offense_val), C = 3)
Y_oh_n_offense_train = convert_to_one_hot(np.array(Yn_offense), C = 4)
Y_oh_n_offense_val = convert_to_one_hot(np.array(Yn_offense_val), C = 4)


In [0]:
# Testing conversion to One hot Vectors
index = 861
for i in range(0,6596):
  if(state_offense[i] == 'hateful_offensive'):
    index = i
    break

print(np.shape(Y_oh_offense_train))
print(state_offense[index], Y_offense[index], "is converted into one hot", Y_oh_offense_train[index])

print(np.shape(Y_oh_n_offense_train))
print(state_offense[index], Yn_offense[index], "is converted into one hot", Y_oh_n_offense_train[index])

<h2> Model using LSTM</h2>

In [0]:
import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import LeakyReLU
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Concatenate, Bidirectional, Add, RepeatVector, Dot, BatchNormalization, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.utils import plot_model

In [0]:
print(tf.__version__)
# K.clear_session()

In [0]:
# Function to convert sentence to indices using the glove vocab
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    
    Arguments:
    X -- array of sentences (strings)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. Sentences longer than this are truncated. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X
    """
    
    m = X.shape[0]                                   # number of training examples
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                                
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words =(X[i].lower()).split(" ")
        
        j = 0        
        for w in sentence_words:
            if j==max_len:
                break
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
                # Increment j to j + 1
                j = j+1
  
    return X_indices

In [0]:
# Test for sentence_to_indices
print("Text: " + text_array[0])
print("Text: " + text_array[1])
test = sentences_to_indices(text_array[:2],word_to_index,10)
print(test)

In [0]:
# Test for rectifying the error on one word at 1800 index in embeddings file. The error was rectified in creating embedding layer.
# u = index_to_word[1800]
# lol = word_to_vec_map[u]
# print(np.shape(lol))
# lol = np.zeros((100,))
# lol[:len(lol)] = word_to_vec_map[u]
# print(np.shape(lol))

In [0]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 100-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary.

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of GloVe word vectors
    
    # Initialize the embedding matrix as a numpy array of zeros of shape
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        enc = word_to_vec_map[word]
        # Handling error in glove 
        if not (np.shape(enc)[0] == emb_dim):
            temp = np.zeros((emb_dim,))
            temp[:len(enc)] = word_to_vec_map[word]
            enc = temp
        # Handling of error complete
        emb_matrix[index] = enc

    # Define Keras embedding layer
    embedding_layer = Embedding(vocab_len, emb_dim, mask_zero = True, trainable = False)

    # Build the embedding layer.
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [0]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

In [0]:
def softmax(x, axis=1):
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')
        


In [0]:
# Global initialisation for Attention layer. The weights need to be same for all iterations.
repeator = RepeatVector(70)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') 
dotor = Dot(axes = 1)

In [0]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    """
    
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas,a])
    
    return context

<h2> Motivation Classifier</h2>

In [0]:
def Motivation_classifier(input_shape,enc_shape, word_to_vec_map, word_to_index):
    """
    Function creating the model's graph.
    
    Arguments:
    input_shape -- shape of the input
    enc_shape -- Size of image features
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 100-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary

    Returns:
    model -- a model instance in Keras
    """
    

    # Define sentence_indices as the input of the graph.
    sentence_indices = Input(shape = input_shape, dtype='int32')
    
    # Embedding layer pretrained with GloVe Vectors
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagating sentence_indices through embedding layer
    embeddings = embedding_layer(sentence_indices)
    
    # Define image_encodings as Input for model
    image_enc = Input(shape = enc_shape, dtype='float32')
    
    # Dense Layers for reducing dimension
    s = Dense(200,activation='relu')(image_enc)
    temp = s

    #Initializing the Initial states of LSTM with images 
    encoder_states = [s, s]
    
    # Propagate the embeddings through an LSTM layer with 200-dimensional hidden state
    embeddings = Dropout(0.2)(embeddings)
    lstm_one = Bidirectional(LSTM(200, return_sequences=True))
    
    X = lstm_one(embeddings)
    X = Dropout(0.4)(X)

    ############__Attention__###############
    context_conv = []

    post_activation_LSTM_cell = LSTM(200, return_state = True, dropout=0.2, recurrent_dropout=0.2)

    for t in range(input_shape[0]):
    
        context = one_step_attention(X, temp)
    
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, s])

        context_conv.append((s))
    ############__Attention__###############

    conv = keras.layers.Concatenate()(context_conv)
    conv = keras.layers.Reshape((input_shape[0],200))(conv)

    X = keras.layers.GRU(64, return_sequences= False, dropout=0.2, recurrent_dropout=0.2)(conv)

    X = BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(X)
    
    # Add dropout with a probability of 0.1
    X = Dropout(0.1)(X)
        
    X = Dense(1,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)

    X = Activation('sigmoid')(X)
    
    # Creating Model instance 
    model = Model(inputs=[sentence_indices, image_enc], outputs=[X], name="motivation_task")
    
    
    return model

In [0]:
model_motivate = Motivation_classifier((70,),(2048,), word_to_vec_map, word_to_index)

In [0]:
model_motivate.compile(loss=['binary_crossentropy'], optimizer="Adam", metrics=['acc'])

In [0]:
# Converting sentences to indices for model input
X_train_indices = sentences_to_indices(X, word_to_index, 70)
print(np.shape(X_train_indices))
X_val_indices = sentences_to_indices(X_val, word_to_index, 70)
print(np.shape(X_val_indices))

In [0]:
# Callbacks for saving weights to pick one with best accuracy
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='/content/memotion_motivation.{epoch:03d}.h5',
                                 monitor='val_acc',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 period=10)

In [0]:
# Finding count of each class
count_motivate = Counter(Y_motivate)
print(count_motivate)
print(count_motivate.keys())
motivational = count_motivate[1]
not_motivational = count_motivate[0]
total = motivational + not_motivational
print("Motivational: ",motivational)
print("Not Motivational: ",not_motivational)
print("Total: ",total)

In [0]:
# class weights calculation
class_weight_motivation = {}
maxi = max(not_motivational, motivational)
weight_for_0 = (maxi / (maxi + not_motivational))
weight_for_1 = (maxi / (maxi + motivational))

class_weight_motivation = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [0]:
history_motivate = model_motivate.fit(
    [X_train_indices,image_encodings],
    [Y_motivate],
    batch_size = 200,
    epochs = 120,
    validation_data = ([X_val_indices,image_encodings_val],[Y_motivate_val]),
    callbacks = [checkpoint],
    class_weight = [class_weight_motivation]
)

In [0]:
history = history_motivate
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.show()


In [0]:
# Loading saved weights
# model_motivate.load_weights('/content/drive/My Drive/Memotion /memotion validation/memotion_motivate.h5')

In [0]:
answer_motivate = model_motivate.predict([X_val_indices,image_encodings_val])

In [0]:
pred_motivate = []

for i in range(0,len(X_val)):
    num = answer_motivate[i]
    if(num>0.5):
      pred_motivate.append(1)
    else:
      pred_motivate.append(0)


In [0]:
con_mat_motivate = tf.math.confusion_matrix(labels=Y_motivate_val, predictions=pred_motivate)
con_mat_motivate = tf.Session().run(con_mat_motivate)
print(con_mat_motivate)

In [0]:
from sklearn.metrics import f1_score
f1_score(Y_motivate_val, pred_motivate, average='micro')

<h2>End of motivation</h2>

<h2>Sarcasm</h2>

In [0]:
K.clear_session()

In [0]:
repeator = RepeatVector(70)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') 
dotor = Dot(axes = 1)

In [0]:
def Sarcasm_classifier(input_shape,enc_shape, word_to_vec_map, word_to_index):
    """
    Function creating the model's graph.
    
    Arguments:
    input_shape -- shape of the input
    enc_shape -- Size of image features
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 100-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary

    Returns:
    model -- a model instance in Keras
    """
    

    # Define sentence_indices as the input of the graph.
    sentence_indices = Input(shape = input_shape, dtype='int32')
    
    # Embedding layer pretrained with GloVe Vectors
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)
    
    # Define image_encodings as Input for model
    image_enc = Input(shape = enc_shape, dtype='float32')
    
    # Dense Layers for reducing dimension
    s = Dense(200,activation='relu')(image_enc)
    temp = s

    #Initializing the hidden states of LSTM with images 
    encoder_states = [s, s]
    
    # Propagate the embeddings through an LSTM layer with 200-dimensional hidden state
    embeddings = Dropout(0.2)(embeddings)
    lstm_one = Bidirectional(LSTM(200, return_sequences=True))
    
    X = lstm_one(embeddings)
    X = Dropout(0.4)(X)

    ############__Attention__###############
    context_conv = []

    post_activation_LSTM_cell = LSTM(200, return_state = True, dropout=0.2, recurrent_dropout=0.2)

    for t in range(input_shape[0]):
    
        context = one_step_attention(X, temp)
            
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, s])
        
        context_conv.append((s))
    ############__Attention__###############

    conv = keras.layers.Concatenate()(context_conv)
    conv = keras.layers.Reshape((input_shape[0],200))(conv)

    X = keras.layers.GRU(64, return_sequences= False, dropout=0.2, recurrent_dropout=0.2)(conv)

    X = BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(X)
    
    # Add dropout with a probability of 0.1
    X = Dropout(0.1)(X)

    X = Dense(3,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)
    Y = Dense(4,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)
    
    # Add a softmax activation
    X = Activation('softmax')(X)
    Y = Activation('softmax')(Y)
    
    # Model instance
    model = Model(inputs=[sentence_indices, image_enc], outputs=[X,Y], name="sarcasm_task")
    
    
    return model

In [0]:
model_sarcasm = Sarcasm_classifier((70,),(2048,), word_to_vec_map, word_to_index)

In [0]:
model_sarcasm.compile(loss=['categorical_crossentropy','categorical_crossentropy'], optimizer="Adam", metrics=['acc'])

In [0]:
# Converting sentence to indices
X_train_indices = sentences_to_indices(X, word_to_index, 70)
print(np.shape(X_train_indices))
X_val_indices = sentences_to_indices(X_val, word_to_index, 70)
print(np.shape(X_val_indices))

In [0]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='/content/memotion_sarcasm.{epoch:03d}.h5',
                                 monitor='val_activation_2_acc',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 period=10)

In [0]:
# Count for each prediction class 
count_sarcasm = Counter(Yn_sarcasm)
print(count_sarcasm)
print(count_sarcasm.keys())
not_sarcastic = count_sarcasm[0]
general = count_sarcasm[1]
twisted_meaning = count_sarcasm[2]
very_twisted = count_sarcasm[3]

total = not_sarcastic + general + very_twisted + twisted_meaning
print("Not sarcastic: ",not_sarcastic)
print("General: ",general)
print("Twisted meaning: ",twisted_meaning)
print("Very twisted: ",very_twisted)
print("Total: ",total)

In [0]:
# class weights for quantification layer
class_weight_sarcasm_4 = {}
maxi = max(not_sarcastic, general, very_twisted, twisted_meaning)
weight_for_0 = (maxi / (maxi + not_sarcastic))
weight_for_1 = (maxi / (maxi + general))
weight_for_2 = (maxi / (maxi + twisted_meaning))
weight_for_3 = (maxi / (maxi + very_twisted))

class_weight_sarcasm_4 = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2, 3: weight_for_3}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))
print('Weight for class 3: {:.2f}'.format(weight_for_3))

In [0]:
# Class weights for classification layer
class_weight_sarcasm_3 = {}
maxi = max(not_sarcastic, general, very_twisted + twisted_meaning)
weight_for_0 = (maxi / (maxi + not_sarcastic))
weight_for_1 = (maxi / (maxi + general))
weight_for_2 = (maxi / (maxi + twisted_meaning + very_twisted))

class_weight_sarcasm_3 = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))

In [0]:
history_sarcasm = model_sarcasm.fit(
    [X_train_indices,image_encodings],
    [Y_oh_sarcasm_train,Y_oh_n_sarcasm_train],
    batch_size = 200,
    epochs = 120,
    validation_data = ([X_val_indices,image_encodings_val],[Y_oh_sarcasm_val,Y_oh_n_sarcasm_val]),
    callbacks = [checkpoint],
    class_weight = [class_weight_sarcasm_3,class_weight_sarcasm_4]
)

In [0]:
history = history_sarcasm
import matplotlib.pyplot as plt
plt.plot(history.history['activation_1_acc'])
plt.plot(history.history['val_activation_1_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train-3', 'Test-3'], loc='lower right')
plt.show()

plt.plot(history.history['activation_1_loss'])
plt.plot(history.history['val_activation_1_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train-3', 'Test-3'], loc='lower right')
plt.show()

# ----------------------------------------------------------------
plt.plot(history.history['activation_2_acc'])
plt.plot(history.history['val_activation_2_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train-4', 'Test-4'], loc='lower right')
plt.show()

plt.plot(history.history['activation_2_loss'])
plt.plot(history.history['val_activation_2_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train-4', 'Test-4'], loc='lower right')
plt.show()

#----------------------------------------------------------------
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.show()

In [0]:
# Loading saved weights
# model_sarcasm.load_weights("/content/drive/My Drive/Memotion /memotion validation/memotion_sarcasm.h5")

In [0]:
# answer_sarcasm[0][i] contains results of classification layer
# answer_sarcasm[1][i] contains results of quantification layer
answer_sarcasm = model_sarcasm.predict([X_val_indices,image_encodings_val])

In [0]:
pred_sarcasm = []
predn_sarcasm = [] # Quantification result
pred2_sarcasm = [] # Classification result

for i in range(0,len(X_val)):
    pred_sarcasm.append(np.argmax(answer_sarcasm[0][i])) # 3 NEURON OUTPUT
    predn_sarcasm.append(np.argmax(answer_sarcasm[1][i])) # 4 NEURON OUTPUT
    
    # Classification Output using 3 neuron output
    num = np.argmax(answer_sarcasm[0][i])
    if(num ==0 ):
      pred2_sarcasm.append(0)
    else:
      pred2_sarcasm.append(1)

In [0]:
con_n_mat_sarcasm = tf.math.confusion_matrix(labels=Yn_sarcasm_val, predictions=predn_sarcasm)
con_n_mat_sarcasm = tf.Session().run(con_n_mat_sarcasm)
print(con_n_mat_sarcasm)

In [0]:
from sklearn.metrics import f1_score
f1_score(Yn_sarcasm_val, predn_sarcasm, average='micro')

In [0]:
con_nf_mat_sarcasm = tf.math.confusion_matrix(labels=Ynf_sarcasm_val, predictions=pred2_sarcasm)
con_nf_mat_sarcasm = tf.Session().run(con_nf_mat_sarcasm)
print(con_nf_mat_sarcasm)

In [0]:
from sklearn.metrics import f1_score
f1_score(Ynf_sarcasm_val, pred2_sarcasm, average='micro')

<h2> End of Sarcasm </h2>

<h2>Humour</h2>

In [0]:
K.clear_session()

In [0]:
repeator = RepeatVector(70)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') 
dotor = Dot(axes = 1)

In [0]:
def Humour_classifier(input_shape,enc_shape, word_to_vec_map, word_to_index):
    """
    Function creating the model's graph.
    
    Arguments:
    input_shape -- shape of the input
    enc_shape -- Size of image features
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 100-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary

    Returns:
    model -- a model instance in Keras
    """
    

    # Define sentence_indices as the input of the graph.
    sentence_indices = Input(shape = input_shape, dtype='int32')
    
    # Embedding layer pretrained with GloVe Vectors
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)
    
    # Define image_encodings as Input
    image_enc = Input(shape = enc_shape, dtype='float32')
    
    # Dense Layers for reducing dimension
    s = Dense(200,activation='relu')(image_enc)
    temp = s

    #Initializing the activations of LSTM with images 
    encoder_states = [s, s]
    
    # Propagate the embeddings through an LSTM layer with 200-dimensional hidden state.
    embeddings = Dropout(0.2)(embeddings)
    lstm_one = Bidirectional(LSTM(200, return_sequences=True))
    
    X = lstm_one(embeddings)
    X = Dropout(0.4)(X)

    ############__Attention__###############
    context_conv = []

    post_activation_LSTM_cell = LSTM(200, return_state = True, dropout=0.2, recurrent_dropout=0.2)

    for t in range(input_shape[0]):
    
        context = one_step_attention(X, temp)
            
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, s])

        context_conv.append((s))
    ############__Attention__###############

    conv = keras.layers.Concatenate()(context_conv)
    conv = keras.layers.Reshape((input_shape[0],200))(conv)

    X = keras.layers.GRU(64, return_sequences= False, dropout=0.2, recurrent_dropout=0.2)(conv)

    X = BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(X)
    
    # Add dropout with a probability of 0.1
    X = Dropout(0.1)(X)

    X = Dense(3,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)
    Y = Dense(4,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    Y = Activation('softmax')(Y)
    
    # Create Model instance.
    model = Model(inputs=[sentence_indices, image_enc], outputs=[X,Y], name="humour_task")
    
    
    return model

In [0]:
model_humour = Humour_classifier((70,),(2048,), word_to_vec_map, word_to_index)

In [0]:
model_humour.compile(loss=['categorical_crossentropy','categorical_crossentropy'], optimizer="Adam", metrics=['accuracy'])

In [0]:
# COnverting sentences to indices
X_train_indices = sentences_to_indices(X, word_to_index, 70)
print(np.shape(X_train_indices))
X_val_indices = sentences_to_indices(X_val, word_to_index, 70)
print(np.shape(X_val_indices))

In [0]:
# Checkpoint callback
checkpoint = ModelCheckpoint(filepath='/content/memotion_humour.{epoch:03d}.h5',
                                 monitor='val_activation_2_acc',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 period=10)

In [0]:
# Count for each class
count_humour = Counter(Yn_humour)
print(count_humour)
print(count_humour.keys())
not_funny = count_humour[0]
funny = count_humour[1]
very_funny = count_humour[2]
hilarious = count_humour[3]

total = not_funny + funny + very_funny + hilarious
print("Not funny: ",not_funny)
print("funny: ",funny)
print("Very funny: ",very_funny)
print("Hilarious: ",hilarious)
print("Total: ",total)

In [0]:
# Class weights for quantification
class_weight_humour_4 = {}
maxi = max(not_funny, funny, very_funny, hilarious)
weight_for_0 = (maxi / (maxi + not_funny))
weight_for_1 = (maxi / (maxi + funny))
weight_for_2 = (maxi / (maxi + very_funny))
weight_for_3 = (maxi / (maxi + hilarious))

class_weight_humour_4 = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2, 3: weight_for_3}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))
print('Weight for class 3: {:.2f}'.format(weight_for_3))

In [0]:
# Class weights for classification
class_weight_humour_3 = {}
maxi = max(not_funny, funny, very_funny + hilarious)
weight_for_0 = (maxi / (maxi + not_funny))
weight_for_1 = (maxi / (maxi + funny))
weight_for_2 = (maxi / (maxi + very_funny + hilarious))

class_weight_humour_3 = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))

In [0]:
history_humour = model_humour.fit(
    [X_train_indices,image_encodings],
    [Y_oh_humour_train,Y_oh_n_humour_train],
    batch_size = 200,
    epochs = 120,
    validation_data = ([X_val_indices,image_encodings_val],[Y_oh_humour_val,Y_oh_n_humour_val]),
    callbacks = [checkpoint],
    class_weight = [class_weight_humour_3,class_weight_humour_4]
)

In [0]:
history = history_humour
import matplotlib.pyplot as plt
plt.plot(history.history['activation_1_acc'])
plt.plot(history.history['val_activation_1_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train-3', 'Test-3'], loc='lower right')
plt.show()

plt.plot(history.history['activation_1_loss'])
plt.plot(history.history['val_activation_1_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train-3', 'Test-3'], loc='lower right')
plt.show()

# ----------------------------------------------------------------
plt.plot(history.history['activation_2_acc'])
plt.plot(history.history['val_activation_2_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train-4', 'Test-4'], loc='lower right')
plt.show()

plt.plot(history.history['activation_2_loss'])
plt.plot(history.history['val_activation_2_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train-4', 'Test-4'], loc='lower right')
plt.show()

#----------------------------------------------------------------
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.show()

In [0]:
# Loading saved weights
# model_humour.load_weights("/content/drive/My Drive/Memotion /memotion validation/memotion_humour.h5")

In [0]:
answer_humour = model_humour.predict([X_val_indices,image_encodings_val])

In [0]:
pred_humour = []
predn_humour = [] # Quantification Result
pred2_humour = [] # Classification result

for i in range(0,len(X_val)):
    pred_humour.append(np.argmax(answer_humour[0][i])) # 3 NEURON OUTPUT
    predn_humour.append(np.argmax(answer_humour[1][i])) # 4 NEURON OUTPUT
    
    # Classification Output using 3 neuron output
    num = np.argmax(answer_humour[0][i])
    if(num == 0):
      pred2_humour.append(0)
    else:
      pred2_humour.append(1)

In [0]:
# Quantification
con_mat_n_humour = tf.math.confusion_matrix(labels=Yn_humour_val, predictions=predn_humour)
con_mat_n_humour = tf.Session().run(con_mat_n_humour)
print(con_mat_n_humour)

In [0]:
from sklearn.metrics import f1_score
f1_score(Yn_humour_val, predn_humour, average='micro')

In [0]:
# Classification
con_mat_nf_humour = tf.math.confusion_matrix(labels=Ynf_humour_val, predictions=pred2_humour)
con_mat_nf_humour = tf.Session().run(con_mat_nf_humour)
print(con_mat_nf_humour)

In [0]:
from sklearn.metrics import f1_score
f1_score(Ynf_humour_val, pred2_humour, average='micro')

<h2> End of humour </h2>

<h2> Offense </h2>

In [0]:
K.clear_session()

In [0]:
repeator = RepeatVector(70)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') 
dotor = Dot(axes = 1)

In [0]:
def Offense_classifier(input_shape,enc_shape, word_to_vec_map, word_to_index):
    """
    Function creating the model's graph.
    
    Arguments:
    input_shape -- shape of the input
    enc_shape -- Size of image features
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 100-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph.
    sentence_indices = Input(shape = input_shape, dtype='int32')
    
    # Embedding layer pretrained with GloVe Vectors
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)
    
    # Define image_encodings as Input for the model
    image_enc = Input(shape = enc_shape, dtype='float32')
    
    # Dense Layers for reducing dimension
    s = Dense(200,activation='relu')(image_enc)
    temp = s

    #Initializing the hidden states of LSTM with images 
    encoder_states = [s, s]
    
    # Propagate the embeddings through an LSTM layer with 200-dimensional hidden state
    embeddings = Dropout(0.2)(embeddings)
    lstm_one = Bidirectional(LSTM(200, return_sequences=True))
    
    X = lstm_one(embeddings)
    X = Dropout(0.4)(X)

    ############__Attention__###############
    context_conv = []

    post_activation_LSTM_cell = LSTM(200, return_state = True, dropout=0.3, recurrent_dropout=0.2)

    for t in range(input_shape[0]):
    
        context = one_step_attention(X, temp)
            
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, s])

        context_conv.append((s))
    ############__Attention__###############

    conv = keras.layers.Concatenate()(context_conv)
    conv = keras.layers.Reshape((input_shape[0],200))(conv)

    X = keras.layers.GRU(64, return_sequences= False, dropout=0.2, recurrent_dropout=0.2)(conv)

    X = BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(X)
    
    # Add dropout with a probability of 0.1
    X = Dropout(0.1)(X)
    
    X = Dense(3,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)
    Y = Dense(4,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    Y = Activation('softmax')(Y)
    
    # Model instance 
    model = Model(inputs=[sentence_indices, image_enc], outputs=[X,Y], name="offense_task")
    
    
    return model

In [0]:
model_offense = Offense_classifier((70,),(2048,), word_to_vec_map, word_to_index)

In [0]:
model_offense.compile(loss=['categorical_crossentropy','categorical_crossentropy'], optimizer="Adam", metrics=['accuracy'])

In [0]:
# Convert sentences to indices
X_train_indices = sentences_to_indices(X, word_to_index, 70)
print(np.shape(X_train_indices))
X_val_indices = sentences_to_indices(X_val, word_to_index, 70)
print(np.shape(X_val_indices))

In [0]:
# Checkpointing weights
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='/content/memotion_offense.{epoch:03d}.h5',
                                 monitor='val_activation_2_acc',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 period=10)

In [0]:
# COunt for each class
count_offense = Counter(Yn_offense)
print(count_offense)
print(count_offense.keys())
not_offensive = count_offense[0]
slight = count_offense[1]
very_offensive = count_offense[2]
hateful_offensive = count_offense[3]

total = not_offensive + slight + very_offensive + hateful_offensive
print("Not offensive: ",not_offensive)
print("slight: ",slight)
print("Very offensive: ",very_offensive)
print("Hateful offensive: ",hateful_offensive)
print("Total: ",total)

In [0]:
# Class weights for quantification
class_weight_offense_4 = {}
maxi = max(not_offensive, slight, very_offensive, hateful_offensive)
weight_for_0 = (maxi / (maxi + not_offensive))
weight_for_1 = (maxi / (maxi + slight))
weight_for_2 = (maxi / (maxi + very_offensive))
weight_for_3 = (maxi / (maxi + hateful_offensive))

class_weight_offense_4 = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2, 3: weight_for_3}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))
print('Weight for class 3: {:.2f}'.format(weight_for_3))

In [0]:
# Class weights for classification
class_weight_offense_3 = {}
maxi = max(not_offensive, slight, very_offensive + hateful_offensive)
weight_for_0 = (maxi / (maxi + not_offensive))
weight_for_1 = (maxi / (maxi + slight))
weight_for_2 = (maxi / (maxi + very_offensive + hateful_offensive))

class_weight_offense_3 = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))

In [0]:
history_offense = model_offense.fit(
    [X_train_indices,image_encodings],
    [Y_oh_offense_train,Y_oh_n_offense_train],
    batch_size = 200,
    epochs = 120,
    validation_data = ([X_val_indices,image_encodings_val],[Y_oh_offense_val,Y_oh_n_offense_val]),
    callbacks = [checkpoint],
    class_weight = [class_weight_offense_3,class_weight_offense_4]
)

In [0]:
history = history_offense
import matplotlib.pyplot as plt
plt.plot(history.history['activation_1_acc'])
plt.plot(history.history['val_activation_1_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train-3', 'Test-3'], loc='lower right')
plt.show()

plt.plot(history.history['activation_1_loss'])
plt.plot(history.history['val_activation_1_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train-3', 'Test-3'], loc='lower right')
plt.show()

# ----------------------------------------------------------------
plt.plot(history.history['activation_2_acc'])
plt.plot(history.history['val_activation_2_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train-4', 'Test-4'], loc='lower right')
plt.show()

plt.plot(history.history['activation_2_loss'])
plt.plot(history.history['val_activation_2_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train-4', 'Test-4'], loc='lower right')
plt.show()

#----------------------------------------------------------------
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.show()

In [0]:
# Loading saved weights
# model_offense.load_weights("/content/drive/My Drive/Memotion /memotion validation/memotion_offense.h5")

In [0]:
answer_offense = model_offense.predict([X_val_indices,image_encodings_val])

In [0]:
pred_offense = []
predn_offense = [] # Quantification results
pred2_offense = [] # Classification results

for i in range(0,len(X_val)):
    pred_offense.append(np.argmax(answer_offense[0][i]))
    predn_offense.append(np.argmax(answer_offense[1][i]))
    num = np.argmax(answer_offense[0][i])
    if(num == 0):
      pred2_offense.append(0)
    else:
      pred2_offense.append(1)


In [0]:
# Quantification
con_mat_n_offense = tf.math.confusion_matrix(labels=Yn_offense_val, predictions=predn_offense)
con_mat_n_offense = tf.Session().run(con_mat_n_offense)
print(con_mat_n_offense)

In [0]:
from sklearn.metrics import f1_score
f1_score(Yn_offense_val, predn_offense, average='micro')

In [0]:
# Classification
con_mat_nf_offense = tf.math.confusion_matrix(labels=Ynf_offense_val, predictions=pred2_offense)
con_mat_nf_offense = tf.Session().run(con_mat_nf_offense)
print(con_mat_nf_offense)

In [0]:
from sklearn.metrics import f1_score
f1_score(Ynf_offense_val, pred2_offense, average='micro')

<h2> End of Offense </h2>

<h2>Subtask A</h2>

In [0]:
K.clear_session()

In [0]:
repeator = RepeatVector(70)
concatenator = Concatenate(axis=-1)
densor1 = Dense(20, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') 
dotor = Dot(axes = 1)

In [0]:
def Mood_classifier(input_shape,enc_shape, word_to_vec_map, word_to_index):
    """
    Function creating the model's graph.
    
    Arguments:
    input_shape -- shape of the input
    enc_shape -- Size of image features
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 100-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary

    Returns:
    model -- a model instance in Keras
    """

    # Define sentence_indices as the input of the graph.
    sentence_indices = Input(shape = input_shape, dtype='int32')
    
    # Embedding layer pretrained with GloVe Vectors
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)
    
    # Define image_encodings as Input for model
    image_enc = Input(shape = enc_shape, dtype='float32')
    
    # Dense Layers for reducing dimension
    s = Dense(200,activation='relu')(image_enc)
    temp = s

    #Initializing the hidden states of LSTM with images 
    encoder_states = [s, s]
    
    # Propagate the embeddings through an LSTM layer with 200-dimensional hidden state
    embeddings = Dropout(0.2)(embeddings)
    lstm_one = Bidirectional(LSTM(200, return_sequences=True))
    
    X = lstm_one(embeddings)
    X = Dropout(0.4)(X)

    ############__Attention__###############
    context_conv = []

    post_activation_LSTM_cell = LSTM(200, return_state = True, dropout=0.2, recurrent_dropout=0.2)

    for t in range(input_shape[0]):
    
        context = one_step_attention(X, temp)
            
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, s])

        context_conv.append((s))
    ############__Attention__###############

    conv = keras.layers.Concatenate()(context_conv)
    conv = keras.layers.Reshape((input_shape[0],200))(conv)

    X = keras.layers.GRU(64, return_sequences= False, dropout=0.2, recurrent_dropout=0.2)(conv)
    
    X = BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(X)
  
    # Add dropout with a probability of 0.1
    X = Dropout(0.1)(X)
  
    X = Dense(3,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)
    Y = Dense(5,activation='elu',kernel_regularizer=keras.regularizers.l2(0.001))(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    Y = Activation('softmax')(Y)
    
    # Model instance
    model = Model(inputs=[sentence_indices, image_enc], outputs=[X,Y], name="sentiment_task")
    
    
    return model

In [0]:
model_mood = Mood_classifier((70,),(2048,), word_to_vec_map, word_to_index)

In [0]:
model_mood.compile(loss=['categorical_crossentropy','categorical_crossentropy'], optimizer="Adam", metrics=['accuracy'])

In [0]:
# Converting sentence to indices
X_train_indices = sentences_to_indices(X, word_to_index, 70)
print(np.shape(X_train_indices))
X_val_indices = sentences_to_indices(X_val, word_to_index, 70)
print(np.shape(X_val_indices))

In [0]:
# Checkpointing weights
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='/content/memotion_sentiment.{epoch:03d}.h5',
                                 monitor='val_activation_3_acc',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 period=10)

In [0]:
# Count of each class
count_mood = Counter(Yn)
print(count_mood)
print(count_mood.keys())
neutral = count_mood[0]
positive = count_mood[1]
very_positive = count_mood[2]
negative = count_mood[3]
very_negative = count_mood[4]

total = neutral + positive + very_positive + negative + very_negative
print("Neutral: ",neutral)
print("Positive: ",positive)
print("Very positive: ",very_positive)
print("Negative: ",negative)
print("Very negative: ",very_negative)
print("Total: ",total)

In [0]:
# Class weights for quantification
class_weight_mood_4 = {}
maxi = max(neutral, positive, very_positive, negative, very_negative)
weight_for_0 = (maxi / (maxi + neutral))
weight_for_1 = (maxi / (maxi + positive))
weight_for_2 = (maxi / (maxi + very_positive))
weight_for_3 = (maxi / (maxi + negative))
weight_for_4 = (maxi / (maxi + very_negative))

class_weight_mood_4 = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2, 3: weight_for_3, 4: weight_for_4}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))
print('Weight for class 3: {:.2f}'.format(weight_for_3))
print('Weight for class 4: {:.2f}'.format(weight_for_4))

In [0]:
# Class weighhts for classification
class_weight_mood_3 = {}
maxi = max(neutral, positive, very_positive, negative, very_negative)
weight_for_0 = (maxi / (maxi + neutral))
weight_for_1 = (maxi / (maxi + positive + very_positive))
weight_for_2 = (maxi / (maxi + negative + very_negative))

class_weight_mood_3 = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))

In [0]:
history_mood = model_mood.fit(
    [X_train_indices,image_encodings],
    [Y_oh_train,Y_oh_n_train],
    batch_size = 256,
    epochs = 150,
    validation_data = ([X_val_indices,image_encodings_val],[Y_oh_val,Y_oh_n_val]),
    callbacks = [checkpoint],
    class_weight = [class_weight_mood_3,class_weight_mood_4]
)

In [0]:
history = history_mood
import matplotlib.pyplot as plt
plt.plot(history.history['activation_1_acc'])
plt.plot(history.history['val_activation_1_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train-3', 'Test-3'], loc='lower right')
plt.show()

plt.plot(history.history['activation_1_loss'])
plt.plot(history.history['val_activation_1_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train-3', 'Test-3'], loc='lower right')
plt.show()

# ----------------------------------------------------------------
plt.plot(history.history['activation_2_acc'])
plt.plot(history.history['val_activation_2_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train-5', 'Test-5'], loc='lower right')
plt.show()

plt.plot(history.history['activation_2_loss'])
plt.plot(history.history['val_activation_2_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train-5', 'Test-5'], loc='lower right')
plt.show()

#----------------------------------------------------------------
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.show()

In [0]:
# Loading saved weights
# model_mood.load_weights("/content/drive/My Drive/Memotion /memotion validation/memotion_mood.h5")

In [0]:
answer_mood = model_mood.predict([X_val_indices,image_encodings_val])

In [0]:
pred_mood = [] # Classification results
predn_mood = [] # Quantification results

temp = 0
for i in range(0,len(X_val)):
    pred_mood.append(np.argmax(answer_mood[0][i]))
    predn_mood.append(np.argmax(answer_mood[1][i]))
    num = np.argmax(answer_mood[0][i])

In [0]:
# Classification results
con_mat_mood = tf.math.confusion_matrix(labels=Y_val, predictions=pred_mood)
con_mat_mood = tf.Session().run(con_mat_mood)
print(con_mat_mood)

In [0]:
from sklearn.metrics import f1_score
f1_score(Y_val, pred_mood, average='micro')

In [0]:
# Quantification results
con_mat_n_mood = tf.math.confusion_matrix(labels=Yn_val, predictions=predn_mood)
con_mat_n_mood = tf.Session().run(con_mat_n_mood)
print(con_mat_n_mood)

In [0]:
from sklearn.metrics import f1_score
f1_score(Yn_val, predn_mood, average='macro')